### https://blog.floydhub.com/multiprocessing-vs-threading-in-python-what-every-data-scientist-needs-to-know/
### https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.Executor

In [ ]:
%load_ext autoreload
%autoreload 2

In [36]:
import numpy as np
import mpmath as mp
import concurrent.futures

In [37]:
# Skip this cell if CSR2D/ is already added under PYHTONPATH 
import os, sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)  

In [102]:
from csr2d.core import psi_s, psi_x, alpha, kappa
from csr2d.dist import lambda_p_Gauss

In [39]:
from mpmath import ellipf, ellipe
from time import sleep, time

# Simple Example

In [6]:
PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]
def is_prime(n):
    if n % 2 == 0:
        return False

    sqrt_n = int(np.floor(np.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

In [9]:
%%time
with concurrent.futures.ProcessPoolExecutor() as executor:
    for prime in executor.map(is_prime, PRIMES):
        print('%s' %  prime)

True
True
True
True
True
False
CPU times: user 36.7 ms, sys: 271 ms, total: 307 ms
Wall time: 978 ms


In [7]:
%%time
list(map(is_prime, PRIMES))

CPU times: user 3.18 s, sys: 0 ns, total: 3.18 s
Wall time: 3.18 s


[True, True, True, True, True, False]

# Testing for psi_x

In [50]:
gamma = 500
beta = (1-1/gamma**2)**(1/2)
rho = 1
sigmax = 10E-6
sigmaz = 10E-6

# Adjust the step size here
# Small steps might greatly increase computation time...

N = 800
dz = 1.0E-4/N
dx = 1.0E-4/N

In [51]:
zvec2 = np.arange(-10*sigmaz, 10*sigmaz, dz)
xvec2 = np.arange(-10*sigmax, 10*sigmax, dx)
zm2, xm2 = np.meshgrid(zvec2, xvec2, sparse=False, indexing='ij')

beta_grid = beta*np.ones(zm2.shape)

In [13]:
%%time
## Direct numpy vectorization 
lala = psi_x(zm2,xm2,beta);

/global/homes/w/wlou1991/CSR2D/csr2d/core.py:83: RuntimeWarning: invalid value encountered in sqrt
  return temp + ( temp**2 - ( zeta(z,x,beta)/3 + nu(x,beta)**2/36 )**3 )**(1/2)


CPU times: user 1.36 s, sys: 11.6 ms, total: 1.37 s
Wall time: 1.37 s


In [33]:
%%time
## Direct mapping
kaka = np.array( list(map(psi_x, zm2, xm2, beta_grid)))

CPU times: user 2min 11s, sys: 1.69 ms, total: 2min 11s
Wall time: 2min 11s


In [76]:
beta_grid.shape

(401, 401)

In [93]:
%%time 
## With Multi-processing "map" function
with concurrent.futures.ProcessPoolExecutor(max_workers = 32) as executor:
    temp = executor.map(psi_s, zm2, xm2, beta_grid)
    result2 = np.array(list(temp))

/global/homes/w/wlou1991/CSR2D/csr2d/core.py:115: RuntimeWarning: divide by zero encountered in true_divide
  arg3 = 2*eta(z,x,beta)/arg1
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: overflow encountered in multiply
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: invalid value encountered in cos
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: invalid value encountered in sin
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))


CPU times: user 1.05 s, sys: 701 ms, total: 1.75 s
Wall time: 2.27 s


In [59]:
%%time 
## With Multi-processing "map" function
with concurrent.futures.ProcessPoolExecutor(max_workers = 32) as executor:
    temp = executor.map(lambda_p_Gauss, zm2, xm2)
    result2 = np.array(list(temp))

CPU times: user 625 ms, sys: 287 ms, total: 912 ms
Wall time: 823 ms


# Writing loop

In [111]:
beta = 0.9999
z = 0.00
x = 0.00
out = (np.cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*np.sin(2*alpha(z,x,beta)))

In [112]:
(kappa(z,x,beta) - beta*(1+x)*np.sin(2*alpha(z,x,beta)))

8.660600514395767e-06

In [110]:
out

-69.28272480685165

In [92]:
gamma = 500
beta = (1-1/gamma**2)**(1/2)
rho = 1
sigmax = 10E-6
sigmaz = 10E-6



for N in np.arange(100,1100,100):
    t1 = time()
    dz = 1.0E-4/N
    dx = 1.0E-4/N
    
    zvec2 = np.arange(-10*sigmaz, 10*sigmaz, dz)
    xvec2 = np.arange(-10*sigmax, 10*sigmax, dx)
    zm2, xm2 = np.meshgrid(zvec2, xvec2, sparse=False, indexing='ij')
    beta_grid = beta*np.ones(zm2.shape)
    
    with concurrent.futures.ProcessPoolExecutor(max_workers = 24) as executor:
        temp = executor.map(psi_s, zm2, xm2, beta_grid)
        result2 = np.array(list(temp))

    print('Time elapsed for N =', N,': ',time()-t1)

Time elapsed for N = 100 :  0.26421046257019043


/global/homes/w/wlou1991/CSR2D/csr2d/core.py:115: RuntimeWarning: divide by zero encountered in true_divide
  arg3 = 2*eta(z,x,beta)/arg1
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: overflow encountered in multiply
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: invalid value encountered in cos
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: invalid value encountered in sin
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))


Time elapsed for N = 200 :  0.32704734802246094


/global/homes/w/wlou1991/CSR2D/csr2d/core.py:115: RuntimeWarning: divide by zero encountered in true_divide
  arg3 = 2*eta(z,x,beta)/arg1
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: overflow encountered in multiply
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: invalid value encountered in cos
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: invalid value encountered in sin
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))


Time elapsed for N = 300 :  0.42162466049194336


...

Time elapsed for N = 800 :  1.4361028671264648


/global/homes/w/wlou1991/CSR2D/csr2d/core.py:115: RuntimeWarning: divide by zero encountered in true_divide
  arg3 = 2*eta(z,x,beta)/arg1
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: overflow encountered in multiply
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: invalid value encountered in cos
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: invalid value encountered in sin
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))


Time elapsed for N = 900 :  1.884047031402588


/global/homes/w/wlou1991/CSR2D/csr2d/core.py:115: RuntimeWarning: divide by zero encountered in true_divide
  arg3 = 2*eta(z,x,beta)/arg1
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: overflow encountered in multiply
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: invalid value encountered in cos
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))
/global/homes/w/wlou1991/CSR2D/csr2d/core.py:26: RuntimeWarning: invalid value encountered in sin
  out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))


Time elapsed for N = 1000 :  2.265371322631836


# Not faster...

In [32]:
%%time
# With Multi-processing
DD = np.arange(0.)
with concurrent.futures.ProcessPoolExecutor() as executor:
    for PSI_X in executor.map(psi_x, zm2, xm2, beta_grid):
        DD = np.append(DD,PSI_X)

CPU times: user 207 ms, sys: 307 ms, total: 514 ms
Wall time: 15.3 s


In [28]:
DD.reshape(zm2.shape);